# GDELT 1.0 Code (skip)

In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'
gdelt_gkg_url = 'http://api.gdeltproject.org/api/v1/gkg_geojson'
# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if str.isdigit(x[0:4])]


In [2]:
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
directory = requests.get(masterListUrl)
results = directory.content.split('\n')

In [233]:
results;

In [234]:
import pandas as pd

pd.options.display.max_rows = 200
# df = pd.DataFrame(data.json())

In [ ]:
df['coords'] = df.features.apply(lambda row: row['geometry']['coordinates'])
df['lat'] = df.features.apply(lambda row: row['geometry']['coordinates'][1])
df['lon'] = df.features.apply(lambda row: row['geometry']['coordinates'][0])
df['name'] = df.features.apply(lambda row: row['properties']['name'])
df['pubdate'] = df.features.apply(lambda row: row['properties']['urlpubtimedate'])
df['urltone'] = df.features.apply(lambda row: row['properties']['urltone'])
df['mentionedNames'] = df.features.apply(lambda row: row['properties']['mentionednames'])
df['mentioinedThemes'] = df.features.apply(lambda row: row['properties']['mentionedthemes'])
df['url'] = df.features.apply(lambda row: row['properties']['url'])

# GDELT 2.0 Access

In [7]:
import requests
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
directory = requests.get(masterListUrl)
clean = directory.content.split('\n')
clean = map(lambda x: x.split(' '),clean)

# Logic for GDELT module

Enter a date
* default is take current time and most recent file
* enter historical date; defaults to no time specificity
    * parse
    * add feature to enter time for historical and pull closest 15 minute file
    
choose a database
*  Select between events, event mentions or gkg

return it as a python or R dataframe
*  use the feather library for Python


## Parameters

In [13]:
# table type = tblType

all = "mentions", "eventsDatabase", "gdelt knowledge graph"
gkg = ["gdelt knowledge graph"]
events = "events" "database"
mentions =  "mentions database"

alls = ['gkg','events']

## Global variables

In [60]:
gkgHeaders = pd.read_csv(
    '../utils/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv',
    delimiter='\t',usecols=['tableId','dataType','Description']
    )
gkgHeaders.tableId.tolist();

eventsDbHeaders = pd.read_csv('../utils/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv',
                         delimiter=',',usecols=['tableId','dataType','Description'])
eventsDbHeaders.tableId.tolist();

mentionsHeaders = pd.read_csv('../utils/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv',
                         delimiter='\t',usecols=['tableId','dataType','Description'])
mentionsHeaders.tableId.tolist();



In [71]:
strings = ['http://data.gdeltproject.org/gdeltv2/20150225234500.gkg.csv.zip',
          'http://data.gdeltproject.org/gdeltv2/20160919070000.mentions.CSV.zip',
          'http://data.gdeltproject.org/gdeltv2/20160919070000.export.CSV.zip']

dbType = re.search(
            '(mentions|export|gkg)',
            strings[2]
        ).group()
dbType

'export'

## Code Pieces and Functions

In [8]:
defaultDateEntry = ""
stringDateEntry = " 2016 09 18"
historicalDateEntry = "2015 02 25"
errorDate = "What in the heck"

In [9]:
date = defaultDateEntry
time = ""

# Date Functionality (Date ranges)

Use the numpy date range functionality to create strings of dates between ranges in a list.  Then, use the dateutil tool to parse those strings into the correct format.  Then run a query for each date, return the dataframe, and concatenate into a single one.  

In [230]:
import numpy as np
np.arange('2016-08-01', '2016-09-16', dtype='datetime64[D]')

array(['2016-08-01', '2016-08-02', '2016-08-03', '2016-08-04',
       '2016-08-05', '2016-08-06', '2016-08-07', '2016-08-08',
       '2016-08-09', '2016-08-10', '2016-08-11', '2016-08-12',
       '2016-08-13', '2016-08-14', '2016-08-15', '2016-08-16',
       '2016-08-17', '2016-08-18', '2016-08-19', '2016-08-20',
       '2016-08-21', '2016-08-22', '2016-08-23', '2016-08-24',
       '2016-08-25', '2016-08-26', '2016-08-27', '2016-08-28',
       '2016-08-29', '2016-08-30', '2016-08-31', '2016-09-01',
       '2016-09-02', '2016-09-03', '2016-09-04', '2016-09-05',
       '2016-09-06', '2016-09-07', '2016-09-08', '2016-09-09',
       '2016-09-10', '2016-09-11', '2016-09-12', '2016-09-13',
       '2016-09-14', '2016-09-15'], dtype='datetime64[D]')

### Pulling Date information

In [11]:
#############################################
# Parse the date
#############################################


from dateutil.parser import parse
import pandas as pd
import numpy as np 
import requests
import datetime



def parse_date(var):
    """Return datetime object from string."""
    
    try:
        return np.where(isinstance(parse(var),datetime.datetime),
                 parse(var),"Error")             
    except:
        return "You entered an incorrect date.  Check your date format."

    
def dateInputCheck(parse_DateVar):
    """Check user input to retrieve date query."""
    
    return np.where(len(parse_DateVar)==0,datetime.datetime.now(),
             parse_date(parse_DateVar)) 


def gdelt_timeString(dateInputVar):
    """Convert date to GDELT string file format for query."""
    
    multiplier = dateInputVar.tolist().minute / 15
    multiple = 15 * multiplier
    queryDate = np.where(
            multiplier > 1,dateInputVar.tolist().replace(
            minute=0, second=0) + datetime.timedelta(
            minutes=multiple),
            dateInputVar.tolist().replace(
            minute=0, second=0,microsecond=0000)
            )
    
    # Check for date equality on historical query
    modifierTip = datetime.datetime.now().replace(
        hour=0,minute=0,second=0,microsecond=0
        ) == queryDate.tolist().replace(
        hour=0,minute=0,second=0,microsecond=0
        )
    
    # Based on modifier, get oldest file for historical query
    queryDate = np.where(
        modifierTip==False,
        queryDate.tolist().replace(
            hour=23,
            minute=45,
            second=00,
            microsecond=0000
            ),queryDate
        )
    
#     print modifierTip
    return queryDate.tolist().strftime("%Y%m%d%H%M%S")

#############################################
# Match parsed date to GDELT master list
#############################################

def match_date(dateString):
    """Return dataframe with GDELT data for matching date"""
    
    masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
    directory = requests.get(masterListUrl)
    results = directory.content.split('\n')
    results = map(lambda x: x.split(' '),results)
    masterListdf = pd.DataFrame(results)
    return masterListdf[
        masterListdf[2].str.contains(
            dateString
            )==True
        ]
    


In [166]:
results = match_date(gdelt_timeString(dateInputCheck(date)))

In [167]:
results

,0,1,2
166641,174299,39ba2ff2f6324fc991c4e1a108156539,http://data.gdeltproject.org/gdeltv2/201609212...
166642,439023,2f07528e827198da2ef4d4edff1eb098,http://data.gdeltproject.org/gdeltv2/201609212...
166643,16871119,2e923aa27d18fed741e9b03b4c5b2d2b,http://data.gdeltproject.org/gdeltv2/201609212...


## Munging Data: Extracting Specific Datasets or all of them

Work with the returned GDELT dataframe.  Specific whether we are pulling the `mentions`, `events`, or `gkg` date for the day or all.  

In [182]:
zippie2 = results.reset_index().ix[1][2]

In [163]:
#############################################
# GDELT data download and extraction
#############################################

from StringIO import StringIO
import pandas as pd
import requests
import zipfile
import re

def downloadAndExtract(gdeltUrl):
    """Downloads and extracts GDELT zips without saving to disk"""
    
    response = requests.get(gdeltUrl, stream=True)
    zipdata = StringIO()
    zipdata.write(response.content)
    gdelt_zipfile = zipfile.ZipFile(zipdata,'r')
    name = re.search('(([\d]{4,}).*)',gdelt_zipfile.namelist()[0]).group().replace('.zip',"")
    data = gdelt_zipfile.read(name)
    gdelt_zipfile.close()
    del zipdata,gdelt_zipfile,name,response
    return pd.read_csv(StringIO(data),delimiter='\t',header=None)
    

def add_header(gdeltUrl):
    """Returns the header rows for the dataframe"""
    
    dbType = re.search(
        '(mentions|export|gkg)',
        gdeltUrl
        ).group()
    
    if dbType == "gkg":
        headers = gkgHeaders.tableId.tolist()
    
    elif dbType == "mentions":
        headers = mentionsHeaders.tableId.tolist()
        
    elif dbType == "export":
        headers = eventsDbHeaders.tableId.tolist()
        
    return headers

In [183]:
zippie2

'http://data.gdeltproject.org/gdeltv2/20160921230000.mentions.CSV.zip'

In [175]:
gdelt_df = downloadAndExtract(zippie)
gdelt_df.columns = add_header(zippie)

In [184]:
gdelt_df2 = downloadAndExtract(zippie2)
gdelt_df2.columns = add_header(zippie2)

In [204]:
combined = gdelt_df.merge(gdelt_df2,how='right',on='GLOBALEVENTID')

In [205]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11767 entries, 0 to 11766
Data columns (total 76 columns):
GLOBALEVENTID                11767 non-null float64
SQLDATE                      2785 non-null float64
MonthYear                    2785 non-null float64
Year                         2785 non-null float64
FractionDate                 2785 non-null float64
Actor1Code                   2550 non-null object
Actor1Name                   2550 non-null object
Actor1CountryCode            1573 non-null object
Actor1KnownGroupCode         63 non-null object
Actor1EthnicCode             31 non-null object
Actor1Religion1Code          55 non-null object
Actor1Religion2Code          13 non-null object
Actor1Type1Code              1220 non-null object
Actor1Type2Code              68 non-null object
Actor1Type3Code              0 non-null float64
Actor2Code                   2005 non-null object
Actor2Name                   2005 non-null object
Actor2CountryCode            1246 non-null obje

In [207]:
combined.tail()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras
11762,581379923.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,879,929,901,1,100,1547,0.000000,NaN,NaN
11763,581387463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,499,477,521,1,100,1547,0.000000,NaN,NaN
11764,581408219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,687,630,665,1,20,813,-9.929078,NaN,NaN
11765,581408220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,704,617,682,0,20,813,-9.929078,NaN,NaN
11766,581408221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,683,596,661,0,20,813,-9.929078,NaN,NaN


In [226]:
# combined.[(combined.Confidence != None) & (combined.MonthYear != None)]
combined.fillna('')[combined.GoldsteinScale <= -5.2]

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras
36,581409211.0,2.01609e+07,201609,2016,2016.7,LBNGOV,TYRE,LBN,,,...,5,1600,1641,1661,0,10,3229,-4.060914,,
37,581409212.0,2.01609e+07,201609,2016,2016.7,LBNGOV,TYRE,LBN,,,...,5,1600,1636,1647,1,30,3229,-4.060914,,
46,581409221.0,2.01609e+07,201609,2016,2016.7,USA,UNITED STATES,USA,,,...,5,1600,-1,1694,0,10,3229,-4.060914,,
55,581409230.0,2.01609e+07,201609,2016,2016.71,LBN,TYRE,LBN,,,...,2,842,791,829,1,60,2749,-7.922912,,
56,581409231.0,2.01609e+07,201609,2016,2016.71,LBN,TYRE,LBN,,,...,2,842,854,829,0,40,2749,-7.922912,,
67,581409242.0,2.01609e+07,201609,2016,2016.71,USA,UNITED STATES,USA,,,...,3,192,499,342,0,20,1777,-9.003215,,
81,581409255.0,2.01609e+07,201609,2016,2016.72,,,,,,...,6,-1,1943,2012,1,100,2229,-3.140097,,
111,581409284.0,2.01609e+07,201609,2016,2016.72,,,,,,...,1,-1,29,39,1,50,3242,-4.504505,,
137,581409305.0,2.01609e+07,201609,2016,2016.72,,,,,,...,10,-1,3980,4010,0,20,7521,-0.867508,,
138,581409305.0,2.01609e+07,201609,2016,2016.72,,,,,,...,9,-1,3719,3741,1,100,6763,-1.218451,,


# Early Pipeline to Write out R Dataframe


Ways to install
```python
pip install feather-format
```

```bash
conda install feather-format -c conda-forge
```


###  **IT WORKS!!!**

In [47]:
import feather
path = 'my_data.feather'
feather.api.write_dataframe(testdf, path)
newtestdf = feather.api.read_dataframe(path)

# Leftovers; Junkyard below here

In [ ]:
results = masterListdf[masterListdf[2].str.contains(gdelt_timeString(dateInputCheck(date)))==True]

In [ ]:
results[2].reset_index().ix[0][2]

In [ ]:
results[results[2].str.contains('gkg')]

In [ ]:
gdelt_timeString(dateInputCheck(date))

In [ ]:
import re
from dateutil.parser import parse
re.search('(([\d]{4,}).*)',clean[20][-1]).group()

In [ ]:
if bool(4>3):
    print "Hello"

In [ ]:
(datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)) == parse("2016 09 18" )

In [ ]:
b = dateutil.parser.parse(re.search('([\d]{4,})',clean[20][-1]).group())

In [ ]:
matchDate = re.search('([\d]{4,})',clean[20][-1]).group()

In [ ]:
def time_change(current,diff):
    date = current.replace(minute=0, second=0) + timedelta(minutes=diff)
    return date.strftime("%Y%m%d%H%M%S")
    

In [ ]:
# pulling most current daily report

import numpy as np
import datetime
from datetime import timedelta

currentTime = datetime.datetime.now()
timeDiff = currentTime.minute / 15 

query = np.where(timeDiff == 1,time_change(currentTime,diff=15),
        np.where(timeDiff == 2, time_change(currentTime,diff=30),
                 np.where(timeDiff == 3, time_change(currentTime,diff=45),
                          time_change(currentTime,diff=0))))

baseUrl = 'http://data.gdeltproject.org/gdeltv2/' + str(query) + '.export.CSV.zip'

In [ ]:
data

In [ ]:
myzipfile.namelist()

In [ ]:

import zipfile


r = requests.get(baseUrl, stream=True)

# with open('gdelt.zip', 'wb') as f:
#     f.write(r.content)
# fh = open('gdelt.zip')
# g = zipfile.ZipFile(fh)
# g.extractall()

from StringIO import StringIO
zipdata = StringIO()
zipdata.write(r.content)
myzipfile = zipfile.ZipFile(zipdata,'r')
data = myzipfile.read(str(query) + '.export.CSV')
gdeltdf = pd.read_csv(StringIO(data),delimiter='\t',header=None)


In [ ]:
gdeltdf.columns=headers.tableId.tolist()

In [ ]:
gdeltdf.SOURCEURL[((gdeltdf.ActionGeo_CountryCode =='SY')|(gdeltdf.ActionGeo_CountryCode =='IZ')) & (gdeltdf.GoldsteinScale < -4)]

In [49]:
text = '''
GLOBALEVENTID	INTEGER	NULLABLE	This is the ID of the event that was mentioned in the article.
EventTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) when the event being mentioned was first recorded by GDELT (the DATEADDED field of the original event record).  This field can be compared against the next one to identify events being mentioned for the first time (their first mentions) or to identify events of a particular vintage being mentioned now (such as filtering for mentions of events at least one week old).
MentionTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) of the current update.  This is identical for all entries in the update file but is included to make it easier to load the Mentions table into a database.
MentionType	INTEGER	NULLABLE	This is a numeric identifier that refers to the source collection the document came from and is used to interpret the MentionIdentifier in the next column.  In essence, it specifies how to interpret the MentionIdentifier to locate the actual document.  At present, it can hold one of the following values:o 1 = WEB (The document originates from the open web and the MentionIdentifier is a fully-qualified URL that can be used to access the document on the web).o 2 = CITATIONONLY (The document originates from a broadcast, print, or other offline source in which only a textual citation is available for the document.  In this case the MentionIdentifier contains the textual citation for the document).o 3 = CORE (The document originates from the CORE archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the CORE website).o 4 = DTIC (The document originates from the DTIC archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the DTIC website).o 5 = JSTOR (The document originates from the JSTOR archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through your JSTOR subscription if your institution subscribes to it).o 6 = NONTEXTUALSOURCE (The document originates from a textual proxy (such as closed captioning) of a non-textual information source (such as a video) available via a URL and the MentionIdentifier provides the URL of the non-textual original source.  At present, this Collection Identifier is used for processing of the closed captioning streams of the Internet Archive Television News Archive in which each broadcast is available via a URL, but the URL offers access only to the video of the broadcast and does not provide any access to the textual closed captioning used to generate the metadata.  This code is used in order to draw a distinction between URL-based textual material (Collection Identifier 1 (WEB) and URL-based non-textual material like the Television News Archive).
MentionSourceName	STRING	NULLABLE	This is a human-friendly identifier of the source of the document.  For material originating from the open web with a URL this field will contain the top-level domain the page was from.  For BBC Monitoring material it will contain “BBC Monitoring” and for JSTOR material it will contain “JSTOR.”  This field is intended for human display of major sources as well as for network analysis of information flows by source, obviating the requirement to perform domain or other parsing of the MentionIdentifier field.
MentionIdentifier	STRING	NULLABLE	This is the unique external identifier for the source document.  It can be used to uniquely identify the document and access it if you have the necessary subscriptions or authorizations and/or the document is public access.  This field can contain a range of values, from URLs of open web resources to textual citations of print or broadcast material to DOI identifiers for various document repositories.  For example, if MentionType is equal to 1, this field will contain a fully-qualified URL suitable for direct access.  If MentionType is equal to 2, this field will contain a textual citation akin to what would appear in an academic journal article referencing that document (NOTE that the actual citation format will vary (usually between APA, Chicago, Harvard, or MLA) depending on a number of factors and no assumptions should be made on its precise format at this time due to the way in which this data is currently provided to GDELT – future efforts will focus on normalization of this field to a standard citation format).  If MentionType is 3, the field will contain a numeric or alpha-numeric DOI that can be typed into JSTOR’s search engine to access the document if your institution has a JSTOR subscription.
SentenceID	INTEGER	NULLABLE	The sentence within the article where the event was mentioned (starting with the first sentence as 1, the second sentence as 2, the third sentence as 3, and so on).  This can be used similarly to the CharOffset fields below, but reports the event’s location in the article in terms of sentences instead of characters, which is more amenable to certain measures of the “importance” of an event’s positioning within an article.
Actor1CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor1 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
Actor2CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor2 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
ActionCharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where the core Action description was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
InRawText	INTEGER	NULLABLE	This records whether the event was found in the original unaltered raw article text (a value of 1) or whether advanced natural language processing algorithms were required to synthesize and rewrite the article text to identify the event (a value of 0).  See the discussion on the Confidence field below for more details.  Mentions with a value of “1” in this field likely represent strong detail-rich references to an event.
Confidence	INTEGER	NULLABLE	Percent confidence in the extraction of this event from this article.  See the discussion in the codebook at http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf
MentionDocLen	INTEGER	NULLABLE	The length in English characters of the source document (making it possible to filter for short articles focusing on a particular event versus long summary articles that casually mention an event in passing).
MentionDocTone	FLOAT	NULLABLE	The same contents as the AvgTone field in the Events table, but computed for this particular article.  NOTE: users interested in emotional measures should use the MentionIdentifier field above to merge the Mentions table with the GKG table to access the complete set of 2,300 emotions and themes from the GCAM system.
MentionDocTranslationInfo	STRING	NULLABLE	This field is internally delimited by semicolons and is used to record provenance information for machine translated documents indicating the original source language and the citation of the translation system used to translate the document for processing.  It will be blank for documents originally in English.  At this time the field will also be blank for documents translated by a human translator and provided to GDELT in English (such as BBC Monitoring materials) – in future this field may be expanded to include information on human translation pipelines, but at present it only captures information on machine translated materials.  An example of the contents of this field might be “srclc:fra; eng:Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0”.  NOTE:  Machine translation is often not as accurate as human translation and users requiring the highest possible confidence levels may wish to exclude events whose only mentions are in translated reports, while those needing the highest-possible coverage of the non-Western world will find that these events often offer the earliest glimmers of breaking events or smaller-bore events of less interest to Western media.o SRCLC. This is the Source Language Code, representing the three-letter ISO639-2 code of the language of the original source material. o ENG.  This is a textual citation string that indicates the engine(s) and model(s) used to translate the text.  The format of this field will vary across engines and over time and no expectations should be made on the ordering or formatting of this field.  In the example above, the string “Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0” indicates that the document was translated using version 2.1.1 of the Moses   SMT platform, using the “MosesCore Europarl fr-en” translation and language models, with the final translation enhanced via GDELT Translingual’s own version 1.0 French translation and language models.  A value of “GT-ARA 1.0” indicates that GDELT Translingual’s version 1.0 Arabic translation and language models were the sole resources used for translation.  Additional language systems used in the translation pipeline such as word segmentation systems are also captured in this field such that a value of “GT-ZHO 1.0 / Stanford PKU” indicates that the Stanford Chinese Word Segmenter   was used to segment the text into individual words and sentences, which were then translated by GDELT Translingual’s own version 1.0 Chinese (Traditional or Simplified) translation and language models.
Extras	STRING	NULLABLE	This field is currently blank, but is reserved for future use to encode special additional measurements for selected material.
'''

In [50]:
from StringIO import StringIO
eventMentions = pd.read_csv(StringIO(text),delimiter='\t',header=None)

In [51]:
eventMentions.columns=['tableId', 'dataType','Empty', 'Description']

In [53]:
eventMentions.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [48]:
eventMentions

,tableId,dataType,Empty,Description
0,GLOBALEVENTID,INTEGER,NULLABLE,This is the ID of the event that was mentioned...
1,EventTimeDate,INTEGER,NULLABLE,This is the 15-minute timestamp (YYYYMMDDHHMMS...
2,MentionTimeDate,INTEGER,NULLABLE,This is the 15-minute timestamp (YYYYMMDDHHMMS...
3,MentionType,INTEGER,NULLABLE,This is a numeric identifier that refers to th...
4,MentionSourceName,STRING,NULLABLE,This is a human-friendly identifier of the sou...
5,MentionIdentifier,STRING,NULLABLE,This is the unique external identifier for the...
6,SentenceID,INTEGER,NULLABLE,The sentence within the article where the even...
7,Actor1CharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...
8,Actor2CharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...
9,ActionCharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...


In [ ]:
gkgdf.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [ ]:
gkgdf.to_csv('GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.csv',sep='\t',index=False,encoding='utf-8')

In [ ]:
headers.to_csv('GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv', index=False,encoding='utf-8')

In [ ]:
import pandas as pd
mentionsdf = pd.read_csv(StringIO(text),delimiter='\t',header=None)
mentionsdf.columns=headers.columns.tolist()